In [1]:
import numpy as np
import math
import sympy as sp
from gurobipy import *
from scipy.optimize import minimize
from scipy.optimize import LinearConstraint
import scipy.optimize
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import timeit
from time import time
import numpy as np
import ctypes

# Supply and demand input

In [3]:
network_file="inputs/network.csv"
demand_file="inputs/demand.csv"
MODnode_match="inputs/MODnodes.csv"
MODoper_coef = "inputs/MODoper_coef.csv"
fleet_size_options = "inputs/fleet_size_options.csv"

In [4]:
## process inputs

M = 10**5

h_bound = 3 ###################################################################

df = pd.read_csv(MODoper_coef)
MODoper = list(df['Operator'])
t_coef = dict()
c_coef = dict()
for i in range(len(MODoper)):
    t_coef[MODoper[i]] = df['t_coef'][i]
    c_coef[MODoper[i]] = df['c_coef'][i]

    
df = pd.read_csv(MODnode_match)
V_coef = list(df['cost'])
MODoper_node = dict()
for i in MODoper:
    MODoper_node[i] = []
for i in range(df.shape[0]):
    MODoper_node[df['operator'][i]].append(df['MOD_node'][i])
MODoper_node_ori = MODoper_node.copy()
MODnode = []
for i in MODoper_node:
    MODnode = MODnode+MODoper_node[i]
MODnode.sort()
MODnode_ori = MODnode.copy()

MODnode_oper = {}
for operr in MODoper:
    for nodee in MODoper_node[operr]:
        MODnode_oper[nodee] = operr
        
df = pd.read_csv(fleet_size_options)
oper_fs_dic = dict()
for i in range(len(MODoper)):
    string = df['fleet_size_options'][i] 
    num_list = string.split(',')
    num_list = [int(num) for num in num_list]
    oper_fs_dic[df['operator'][i]] = num_list

MODnode_h = []
V_coef_h = dict()
for i in MODnode:
    operr = MODnode_oper[i]
    h_num = len(oper_fs_dic[operr])
    for k in range(1,h_num+1):
        MODnode_h.append(i*10+k)
        V_coef_h[i*10+k] = V_coef[MODnode.index(i)]
MODnode = MODnode_h
V_coef = V_coef_h

df = pd.read_csv(network_file)
df = df.dropna()
MODnode_mat = pd.read_csv(MODnode_match)
MODnode_mat = MODnode_mat.dropna()

df_fixed = df.loc[df['link type'] == 'fixed']
df_fixed['fleet_size'] = 0
df_walk = df.loc[df['link type'] == 'transfer']
df_walk['fleet_size'] = 0
df_fixed = pd.concat([df_fixed, df_walk], ignore_index=True)

df_mod_ori = df.loc[df['link type'] == 'MOD']
for ind in range(df_mod_ori.shape[0]):
    df_mod_ = df_mod_ori.iloc[ind].to_frame().T
    operr = df_mod_['operator'].values[0]
    for i in range(len(oper_fs_dic[operr])):
        df_mod = df_mod_.copy()
        df_mod['fleet_size'] = oper_fs_dic[operr][i]
        df_mod['from_node'] = df_mod_ori.iloc[ind]['from_node']*10 + i+1
        df_mod['to_node'] = df_mod_ori.iloc[ind]['to_node']*10 + i+1
        df_fixed = pd.concat([df_fixed,df_mod], ignore_index=True)
    
df_trans = pd.DataFrame(columns=df_fixed.columns)
for j in range(h_bound):###################################################################
    for i in range(MODnode_mat.shape[0]):
        data = [[0,MODnode_mat.iloc[i]['fixed_node'],MODnode_mat.iloc[i]['MOD_node']*10+j+1,0,0,0,MODnode_mat.iloc[i]['operator'],'Transfer to MOD',j+1]]
        df_trans_append = pd.DataFrame(data, columns=df_fixed.columns)
        df_trans = pd.concat([df_trans, df_trans_append], ignore_index=True)

        data = [[0,MODnode_mat.iloc[i]['MOD_node']*10+j+1,MODnode_mat.iloc[i]['fixed_node'],0,0,0,MODnode_mat.iloc[i]['operator'],'Transfer from MOD',j+1]]
        df_trans_append = pd.DataFrame(data, columns=df_fixed.columns)    
        df_trans = pd.concat([df_trans, df_trans_append], ignore_index=True)
df_all = pd.concat([df_fixed, df_trans], ignore_index=True)


#Input demand
df11 = pd.read_csv(demand_file)
df11 = df11[['user_number','from_node','to_node','demand_amount','utility']]
df11 = df11.dropna()
df_=df11[df11.from_node != df11.to_node]
orig=list(df_["from_node"])
dest=list(df_["to_node"])
d=list(df_["demand_amount"])
utility = list(df_["utility"])

#Dummy links       
df_dummy = pd.DataFrame(columns=df_fixed.columns)
for j in range(len(d)):
    data = [[0,orig[j],dest[j],utility[j],0,M,0,'Dummy link',0]]
    df_dummy_append = pd.DataFrame(data, columns=df_fixed.columns)
    df_dummy = pd.concat([df_dummy, df_dummy_append], ignore_index=True)
df_all = pd.concat([df_all, df_dummy], ignore_index=True)

#Input the whole network
df_all = df_all.reset_index()
df_all = df_all.drop(columns = ['index'])
df_all['link number'] = list(range(1, df_all.shape[0]+1))
df2 = df_all.copy()
links=len(list(df2['from_node'])) # number of links
link={}
dummy_link = []
for i in range(0,links):
    new_result={'start':'','end':'','operating cost':'','capacity':'','travel cost':'','operator':'','fleet size':''}
    link[i]=new_result
    link[i]['start']=df2['from_node'][i]
    link[i]['end']=df2['to_node'][i]
    link[i]['travel cost']=df2['travel cost'][i]
    link[i]['operating cost']=df2['operating cost'][i]
    link[i]['capacity']=df2['capacity'][i]
    link[i]['operator']=df2['operator'][i]
    link[i]['fleet size']=df2['fleet_size'][i]
    if df2['link type'][i] == 'Dummy link':
        ab = list(zip(orig, dest))
        od = ab.index((df2['from_node'][i], df2['to_node'][i]))
        link[i]['travel cost']=utility[od]
        dummy_link.append(i)
        
nodes=list(set().union(list(df2['to_node']), list(df2['from_node'])))
oper_list=set(list(df2['operator']))

demand=len(orig)
k={}
for i in nodes:
    k[i]=0 
    for s in range(demand):
        if i==dest[s]:
            k[i]-=d[s]
        if i==orig[s]:
            k[i]+=d[s]

MODoper_node_ori = MODoper_node.copy()

MODoper_node_h = dict()
for i in MODoper_node:
    MODoper_node_h[i] = []   
for i in range(h_bound):###################################################################
    for nodee in MODoper_node:
        for k in MODoper_node[nodee]:
            MODoper_node_h[nodee].append(k*10+i+1)            
MODoper_node = MODoper_node_h.copy()


MODnode_oper_h = {}
for nodee in MODnode_oper:
    for i in range(h_bound):###################################################################
        MODnode_oper_h[nodee*10+i+1] = MODnode_oper[nodee]
        
MODnode_oper = MODnode_oper_h.copy()


#FIXED network
df2 = df_all.loc[df_all['link type'] == 'fixed']
links_fixed=len(list(df2['from_node'])) # number of links
link_fixed={}
for i in range(0,links_fixed):
    new_result={'start':'','end':'','operating cost':'','capacity':'','travel cost':'','operator':'','fleet size':''}
    link_fixed[i]=new_result
    link_fixed[i]['start']=df2['from_node'][i]
    link_fixed[i]['end']=df2['to_node'][i]
    link_fixed[i]['travel cost']=df2['travel cost'][i]
    link_fixed[i]['operating cost']=df2['operating cost'][i]
    link_fixed[i]['capacity']=df2['capacity'][i]
    link_fixed[i]['operator']=df2['operator'][i]
    link_fixed[i]['fleet size']=df2['fleet_size'][i]

#transfer links
df2 = df_all.loc[df_all['link type'] == 'transfer']
links_trans=len(list(df2['from_node'])) # number of links
link_trans={}
for i in list(df2.index):
    new_result={'start':'','end':'','operating cost':'','capacity':'','travel cost':'','operator':'','fleet size':''}
    link_trans[i]=new_result
    link_trans[i]['start']=df2['from_node'][i]
    link_trans[i]['end']=df2['to_node'][i]
    link_trans[i]['travel cost']=df2['travel cost'][i]
    link_trans[i]['operating cost']=df2['operating cost'][i]
    link_trans[i]['capacity']=df2['capacity'][i]
    link_trans[i]['operator']=df2['operator'][i]
    link_trans[i]['fleet size']=df2['fleet_size'][i]

#Mapping of the access links and exit links
df2 = df_all.loc[df_all['link type'] == 'Transfer to MOD']
links_Acce=len(list(df2['from_node'])) # number of links
link_Acce={}
for i in list(df2.index):
    new_result={'start':'','end':'','operator':''}
    link_Acce[i]=new_result
    link_Acce[i]['start']=df2['from_node'][i]
    link_Acce[i]['end']=df2['to_node'][i]
    link_Acce[i]['operator']=df2['operator'][i]
    link_Acce[i]['fleet size']=df2['fleet_size'][i]


df2 = df_all.loc[df_all['link type'] == 'Transfer from MOD']
links_Egre=len(list(df2['from_node'])) # number of links
link_Egre={}
for i in list(df2.index):
    new_result={'start':'','end':'','operator':''}
    link_Egre[i]=new_result
    link_Egre[i]['start']=df2['from_node'][i]
    link_Egre[i]['end']=df2['to_node'][i]
    link_Egre[i]['operator']=df2['operator'][i]
    link_Egre[i]['fleet size']=df2['fleet_size'][i]


#Mapping MOD operators to the access links
df2 = df_all.loc[df_all['link type'] == 'MOD']
df2 = df2.reset_index()
MOD_operator = list(np.unique(np.array(df2['operator'])))
df3 = df_all.loc[df_all['link type'] == 'Transfer to MOD']
MODoper_Acce = {}
for i in MOD_operator:
    MODoper_Acce[i] = []
for i in link_Acce:
    for oper in MOD_operator:
        if link_Acce[i]['operator'] == oper:
            MODoper_Acce[oper].append([link_Acce[i]['start'],link_Acce[i]['end']])
            
#MOD nodes
df2 = df_all.loc[df_all['link type'] == 'MOD']
from_nodes = np.array([df2['from_node']])
to_nodes = np.array([df2['to_node']])
MOD_nodes = np.hstack((from_nodes,to_nodes))
MOD_nodes = np.unique(MOD_nodes)


#MOD LINKS
df2 = df_all.loc[df_all['link type'] == 'MOD']
links_MOD=len(list(df2['from_node'])) # number of links
link_MOD={}
for i in df2.index:
    new_result={'start':'','end':'','operating cost':'','capacity':'','travel cost':'','operator':'','fleet size':''}
    link_MOD[i]=new_result
    link_MOD[i]['start']=df2['from_node'][i]
    link_MOD[i]['end']=df2['to_node'][i]
    link_MOD[i]['travel cost']=df2['travel cost'][i]
    link_MOD[i]['operating cost']=df2['operating cost'][i]
    link_MOD[i]['capacity']=df2['capacity'][i]
    link_MOD[i]['operator']=df2['operator'][i]
    link_MOD[i]['fleet size']=df2['fleet_size'][i]

link_from_to = dict()
for i in link:
    if i not in dummy_link:
        link_from_to[(link[i]['start'], link[i]['end'])] = i  

        
# Assignment matching problem
outflow=dict()
inflow=dict()
outflow_link=dict()
inflow_link=dict()
lst=[]
lst2=[]
for i in range(links):
    lst2.append((link[i]['start'],link[i]['end']))
    for s in range(demand):
        lst.append((link[i]['start'],link[i]['end'],s)) 
    a=link[i]['start']
    b=link[i]['end']
    if b in inflow:
        inflow[b].append(link[i]['start'])
        inflow_link[b].append(i)
    else:
        inflow[b]=[link[i]['start']]
        inflow_link[b]=[i]
    if a in outflow:
        outflow[a].append(link[i]['end'])
        outflow_link[a].append(i)
    else:
        outflow[a]=[link[i]['end']]
        outflow_link[a]=[i]
for i in nodes:
    if i not in inflow:
        inflow[i]={}
        inflow_link[i]=[]
    if i not in outflow:
        outflow[i]={}
        outflow_link[i]=[]

node = []
for i in outflow_link:
    node.append(i)

    
df1 = df_all.loc[df_all['link type'] == 'Transfer to MOD']
df2 = df_all.loc[df_all['link type'] == 'Transfer from MOD']
transin_oper = dict()
for i in range(len(link_fixed)+len(link_trans)+len(link_MOD),len(link_fixed)+len(link_trans)+len(link_MOD)+len(link_Acce)+len(link_Egre),2):
    transin_oper[i] = df1['operator'][i]


oper_transin = dict()
for j in range(min(MODoper),max(MODoper)+1):
    oper_transin[j] = []
for i in range(len(link_fixed)+len(link_trans)+len(link_MOD),len(link_fixed)+len(link_trans)+len(link_MOD)+len(link_Acce)+len(link_Egre),2):
    oper = df1['operator'][i]
    for j in range(min(MODoper),max(MODoper)+1):
        if oper == j:
            heiheihei = oper_transin[j]
            heiheihei.append(i)
            oper_transin[j] = heiheihei

kkk={}
for s in range(len(d)):
    for i in nodes:
        kkk[i,s]=0
        if i==dest[s]:
            kkk[i,s]=-d[s]
        if i==orig[s]:
            kkk[i,s]=d[s]
        if orig[s]==dest[s]:
            kkk[i,s]=0
            
def bbb_in_aaa(aaa,bbb):
    result = False
    if len(aaa) >= len(bbb):
        for index, value in enumerate(aaa):
            if value == bbb[0]:
                result = True
                for i in range(1, len(bbb)):
                    if aaa[index+i] != bbb[i]:
                        result = False
    return result

def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3


oper_cost = []
for i in range(len(link_fixed)):
    oper_cost.append(link_fixed[i]['operating cost'])
    
cap = []
for i in range(len(link_fixed)):
    cap.append(link_fixed[i]['capacity'])
    
b = np.zeros((len(orig),len(link_fixed)))
for k in range(len(orig)):
    for links in range(len(link_fixed)):
        b[k,links] = min(cap[links],d[k])
        
a_alllink = list(link.keys())
a_fixedlink = list(link_fixed.keys())
a_translink = list(link_trans.keys())
a_MODlink = list(link_MOD.keys())
a_transinlink = list(link_Acce.keys())
a_transoutlink = list(link_Egre.keys())

# Set parameters

In [5]:
num_iter = 8000
tolerance = 0.05
PRINT = 0
FW_tol = 10**-2
FW_conver_times = 5

# Import packages

In [6]:
from ctypes import cdll
lib = cdll.LoadLibrary('./foo.so')

cpp_dijkstra = lib.cpp_dijkstra

# _doublepp = np.ctypeslib.ndpointer(dtype=np.uintp, ndim=1, flags='C') 
cpp_dijkstra.argtypes = [
    np.ctypeslib.ndpointer(dtype=ctypes.c_int, ndim=1),
    ctypes.c_int,
    np.ctypeslib.ndpointer(dtype=ctypes.c_int, ndim=1),
    np.ctypeslib.ndpointer(dtype=ctypes.c_int, ndim=1),
    ctypes.c_int
]

from MaaS_game_link_fare_systemopt import b_and_b, rank_by_obj, network_branch, obj_yv, single_branch, Dijkstra_cpp, flow_yv_finding, subgrad_opt, FW, obj, Dijkstra, solve_for_Y_vbounded, find_alpha_der_0
from Cost_allo_sub_link_systemopt import pre_process_intsol, cost_allo_prep, cost_allo, subsidy_stablize, cost_allo_sub

# Solve the MaaS assignment game

In [7]:
%%time

flows_link_best, flows_path_best, y_best, v_best, obj_best, ttl_cost_best, subsidy_obj_best, dual_p_best, path_set_best, p_user_opt_best, u_user_opt_best, p_oper_opt_best, u_oper_opt_best, S_path_best = b_and_b(PRINT, num_iter, tolerance, FW_tol, FW_conver_times, 
                                                                                             t_coef, c_coef, d, orig, dest, cap, b, oper_cost, V_coef, utility, 
                                                                                             link, link_fixed, link_trans, link_Acce, link_MOD, link_Egre, link_from_to,
                                                                                             a_alllink, a_MODlink, a_fixedlink, a_translink, 
                                                                                             transin_oper, MOD_nodes, nodes, MODnode_oper, 
                                                                                             MODnode_ori, MODoper_node_ori, MODnode, MODoper, MODoper_Acce,
                                                                                             outflow_link, inflow_link, dummy_link, oper_list)
    

Root node solving......(node 1 )
Subgradient opt number of iterations: 3058
Academic license - for non-commercial use only - expires 2024-10-18
Using license file /Users/bingqing/gurobi.lic
Root node solved. Obj = 11279.616143862764 Time =  0.0 min 27.320069789886475 sec
Node selected: 
y: [1.0, 0.0, 0.1980546646876373, 0.49457170305828774, 0.29882327742745474, 0.0, 0.0, 0.0, 0.0]
v: [0.0031966491899165835, 0.0048255617807298826, 0.0, 0.0012186898625523106, 0.0005976169354738989, 0.0, 0.0005199258812806986, 0.0, 0.0, 0.0005199258812806986, 0.0, 0.0, 0.0020813719232719804, 0.0, 0.0, 0.0020813719232719804, 0.0, 0.0, 0.0024898585290796063, 0.004227944845255984, 0.0]
Fixed link selected to branch: 2 0.1980546646876373
----- Branch 1 (y - 1) -----
y_1: [2]
y_0: []
v_1: []
v_0: []
Branch solving...... (node 2 )
Subgradient opt number of iterations: 2463
Branch solved. obj_yv_value =  11440.168592336822 Time =  0.0 min 22.534530878067017 sec
NOT INTEGER SOL & better than current best! saved i

Subgradient opt number of iterations: 5795
Branch solved. obj_yv_value =  11844.66446685613 Time =  1.0 min 37.99057602882385 sec
NOT INTEGER SOL & better than current best! saved in queue
obj_yv_value =  11844.66446685613 ttl_cost_best =  inf
----- Branch 2 (v - 0) -----
y_1: [2, 3]
y_0: [4]
v_1: []
v_0: [71, 72]
Branch solving......(node 17 )
Subgradient opt number of iterations: 1032
Branch solved. obj_yv_value =  11846.083742514229 Time =  0.0 min 7.85080099105835 sec
NOT integer sol & better than current best! saved in queue
obj_yv_value =  11846.083742514229 ttl_cost_best =  inf
Queue length =  9
---------------
Node selected: 
y: [1.0, 0.0, 1, 1, 0, 0.0, 0.0, 0.0, 0.0]
v: [1, 0, 0, 1, 0, 0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0018958226526861232, 0.0, 0.0, 0.0018958226526861232, 0.0, 0.0, 0.0021675134662451118, 0, 0]
MOD node selected to branch: 111 0.0018958226526861232
----- Branch 1 (v - 1) -----
y_1: [2, 3]
y_0: [4]
v_1: [71, 81, 111]
v_0: [72, 73, 82, 83, 112, 113, 122, 123, 13

Subgradient opt number of iterations: 833
Branch solved. obj_yv_value =  11851.080021498708 Time =  0.0 min 2.863196849822998 sec
Number of Paths: 3
[41, 17, 52]
[41, 11, 44, 47, 35, 54]
[41, 11, 44, 47, 35, 54]
[41, 11, 44, 10, 2]
[5, 6]
[5, 6]
Read LP format model from file model_infea.ilp
Reading time = 0.00 seconds
: 1 rows, 0 columns, 0 nonzeros
Infeasible path: []
Unstable path: []
Alternative path: []
Queue length =  2
---------------
Node selected: 
y: [1.0, 0.0, 1, 1, 0, 0.0, 0.0, 0.0, 0.0]
v: [1, 0, 0, 0, 0, 0, 0, 0.0, 0.0, 0.0, 0.0, 0.0, 0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.003, 0, 0]
MOD node selected to branch: 131 0.003
----- Branch 1 (v - 1) -----
y_1: [2, 3]
y_0: [4]
v_1: [71, 131]
v_0: [72, 73, 81, 82, 83, 91, 111, 132, 133]
Branch solving......(node 32 )
Subgradient opt number of iterations: 3927
Branch solved. obj_yv_value =  11845.875 Time =  0.0 min 18.564302682876587 sec
Number of Paths: 3
[]
[41, 11, 44, 10, 2]
[5, 6]
[5, 6]
Cost Allocation feasible!
STABLE INTEGER SOL!

In [91]:
# output csvs

os.mkdir('outputs')

df = pd.DataFrame(columns = ["origin", "destination", "start", "end", "operator", "link_flow_of_the_od"])
for od in range(len(d)):
    for i in range(flows_link_best.shape[1]):
        if i in a_transinlink:
            pass
        elif i in a_transoutlink:
            pass
        elif i in a_MODlink:
            aaa = [orig[od], dest[od], int(link[i]['start']/10), int(link[i]['end']/10), link[i]['operator'], flows_link_best[od, i]]
            df.loc[len(df)] = aaa 
        elif i in dummy_link:
            pass
        else:
            aaa = [orig[od], dest[od], link[i]['start'], link[i]['end'], link[i]['operator'], flows_link_best[od, i]]
            df.loc[len(df)] = aaa
df.to_csv('outputs/link_flows_od.csv', index=False)



df = pd.DataFrame(columns = ["origin", "destination", "path", "path_flow", "subsidy_per_person"])
count = 0
for od in range(len(d)):
    count_ = 0
    for i in path_set_best[od]:
        i_ = [orig[od]]
        for ll in i:
            en = link[ll]['end']
            if en in MODnode:
                i_.append(int(en/10))
            else:
                i_.append(en)  
        if S_path_best!= []:
            aaa = [orig[od], dest[od], i_, flows_path_best[count], S_path_best[od, count_]]
        else:
            aaa = [orig[od], dest[od], i_, flows_path_best[count], 0]
        df.loc[len(df)] = aaa
        count += 1
        count_ += 1
df.to_csv('outputs/path_flows_od.csv', index=False)

            
df = pd.DataFrame(columns = ["start", "end", "operator", "operated_or_not"]) 
count = 0
for i in link_fixed:
    aaa = [link[i]["start"], link[i]["end"], link[i]['operator'], y_best[count]]
    df.loc[len(df)] = aaa
    count += 1
df.to_csv('outputs/fixed_route_operation_decisions.csv', index=False)


df = pd.DataFrame(columns = ["node", "operator", "fleet_size", "chosen_or_not"]) 
count = 0
for i in MODnode:
    oper = MODnode_oper[i]
    h_id = i%10-1
    hh = oper_fs_dic[oper][h_id]
    aaa = [int(i/10), MODnode_oper[i], hh, v_best[count]]
    df.loc[len(df)] = aaa
    count += 1
df.to_csv('outputs/MOD_operation_decisions.csv', index=False)

df = pd.DataFrame(columns = ["start", "end", "operator", "buyer_optimal_fare", "seller_optimal_fare"])
for i in p_user_opt_best:
    if i[1] != 0 and (i[0] not in a_transinlink) and (i[0] not in a_transoutlink):
        sss = link[i[0]]['start']
        eee = link[i[0]]['end']
        if i[0] in a_MODlink:
            sss = int(sss/10)
            eee = int(eee/10)  
        aaa = [sss, eee, i[1], p_user_opt_best[i], p_oper_opt_best[i]]
        df.loc[len(df)] = aaa 
df.to_csv('outputs/fares.csv', index=False)

df = pd.DataFrame(columns = ["origin", "destination", "user_surplus_per_person_buyer_optimal", "user_surplus_per_person_seller_optima"])
for od in range(len(d)):
    if od not in u_user_opt_best:
        aaa = [orig[od], dest[od], 0, u_oper_opt_best[od]]
    if od not in u_oper_opt_best:
        aaa = [orig[od], dest[od], u_user_opt_best[od], 0]
    else:
        aaa = [orig[od], dest[od], u_user_opt_best[od], u_oper_opt_best[od]]
    df.loc[len(df)] = aaa
df.to_csv('outputs/user_surplus.csv', index=False)



In [97]:
df = pd.DataFrame(columns = ["node", "operator", "fleet_size", "chosen_or_not"]) 
count = 0
for i in MODnode:
    oper = MODnode_oper[i]
    h_id = i%10-1
    hh = oper_fs_dic[oper][h_id]
    aaa = [int(i/10), MODnode_oper[i], hh, v_best[count]]
    df.loc[len(df)] = aaa
    count += 1
df.to_csv('outputs/MOD_operation_decisions.csv', index=False)

df

,node,operator,fleet_size,chosen_or_not
0,7.0,7.0,1.0,1.0
1,7.0,7.0,2.0,0.0
2,7.0,7.0,3.0,0.0
3,8.0,7.0,1.0,0.0
4,8.0,7.0,2.0,0.0
5,8.0,7.0,3.0,0.0
6,9.0,8.0,1.0,0.0
7,9.0,8.0,2.0,0.0
8,9.0,8.0,3.0,0.0
9,10.0,8.0,1.0,0.0
